# Data Ingestion
Ingest knowledge to vector database for semantic search. We will ingest data from `qa_samples.csv`.

*Note*
- You must grant this notebook instance with access of SageMaker `huggingface-inference-eb` invocation access
- You must grant this notebook instance with access of `VectorDBMasterUserCredentials` and `OpenSearchHostURL` in SecretsManager read access


In [18]:
import pandas as pd

qas = pd.read_csv('qa_samples.csv', usecols = ['question', 'answers'])

qas

,question,answers
0,what are the main quality control methods for ...,The main quality control methods for machined ...
1,What are the common casting defects in foundry?,Some common casting defects in foundry include...
2,What are the main causes of low productivity ...,The main causes of low productivity in assembl...
3,What are some key points for improving qualit...,Some key points for improving quality in manuf...
4,What are some common causes of accidents in i...,Common causes of accidents in industrial workp...
5,What are some key factors to consider for pla...,Some key factors for plant layout and material...
6,What are some common causes of equipment fail...,Common causes of equipment failure include:\n\...
7,What are some key benefits of using automatio...,Benefits of automation in manufacturing includ...
8,What types of lubricants are commonly used in...,Common types of industrial machine lubricants ...
9,What are some key steps when troubleshooting e...,Key troubleshooting steps include:\n\n- Identi...


## Ingest into Vector Database

### Create Opensearch Index

We will create index with 1 vector column `answers_vector` for semantic search.

In [26]:
# The name of index
import boto3, json
import sagemaker
import requests

sm_client = boto3.client('secretsmanager')
def get_auth():
    user_json = sm_client.get_secret_value(SecretId='VectorDBMasterUserCredentials')['SecretString']
    data= json.loads(user_json)
    return (data.get('username'), data.get('password'))

def get_host():
    host_json = sm_client.get_secret_value(SecretId='OpenSearchHostURL')['SecretString']
    data= json.loads(host_json)
    es_host_name = data.get('host')
    es_host_name = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name # cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
    return f'https://{es_host_name}'

awsauth = get_auth()
host = get_host()

# create index
index_name = 'semantic_search_knowledge_index'
v_dimension = 768 # Embbeding vector dimension

headers = { "Content-Type": "application/json" }

payloads = {
    "settings": {
       "index.knn": True,
        "knn.space_type": "l2"
   },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": v_dimension,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {
                        "ef_construction": 256,
                        "m": 32
                    }
                }
            },
            "question": {
                "type": "text"
            },
            "answers": {
                "type": "text"
            }
        }
    }
}

# Create Index
r = requests.put(host+index_name, auth=awsauth, headers=headers, json=payloads)

### Remove the Opensearch Index(Optinal)

In [25]:
## You can remove the index for recreation
requests.delete(host+index_name, auth=awsauth, headers=headers)

<Response [404]>

### Load from prepared doc

Read from csv file

### Ingest data with embeddings
For each record in 'qa_smaples.csv' file,
- Firstly, get inference from embeddings.
- Secondly, post data to vector db

In [11]:
!pip install -q tqdm

In [28]:
import json

endpoint_name = 'RAGSearchWithLLMEndpoint' # The name of embbeding model endpoint
client = boto3.client('sagemaker-runtime')

url = host+'_bulk'
def generate_vector(sentence):
    try:
        sentence = sentence if len(sentence) < 400 else sentence[:400]
        response = client.invoke_endpoint(
                        EndpointName=endpoint_name,
                        Body=json.dumps({'inputs':[sentence]}),
                        ContentType='application/json',
                    )
        vector = json.loads(response['Body'].read())
        return vector[0][0][0]
    except Exception as e:
        print(e)
        return [-1000 for _ in range(v_dimension)]


In [22]:
from tqdm import tqdm
from time import sleep

def import_single_row(payload):
    question_vector = generate_vector(payload['question'])
    payload['question_vector'] = question_vector
    
    first = json.dumps({ "index": { "_index": index_name} }, ensure_ascii=False) + "\n"
    second = json.dumps(payload, ensure_ascii=False) + "\n"
    payloads = first + second
    r = requests.post(url, auth=awsauth, headers=headers, data=payloads.encode()) # requests.get, post, and delete have similar syntax


def import_data(json_array):
    for payload in tqdm(json_array):
        import_single_row(payload)
        sleep(0.01)

In [29]:
import csv
import json

json_array=[]
with open('qa_samples.csv', encoding = 'utf-8') as csv_file_handler:
    csv_reader = csv.DictReader(csv_file_handler)
    for row in csv_reader:
        json_array.append(row)
        
import_data(json_array)

100%|██████████| 19/19 [00:02<00:00,  8.26it/s]
